In [1]:
# Importing Libraries
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# Test 01 - No Road line Detection/ Video playing fast

def processFrame(frame, metersPerPixel):
    # Same steps as with the image: Convert to HSV, detect green lines, and measure road width
    hsvFrame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    lowerGreen = np.array([36, 25, 25])
    upperGreen = np.array([86, 255, 255])
    
    # Create a mask for green color
    greenMask = cv2.inRange(hsvFrame, lowerGreen, upperGreen)
    
    # Find contours in the mask
    contours, _ = cv2.findContours(greenMask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    leftX = []
    rightX = []
    height = frame.shape[0]
    
    for contour in contours:
        for point in contour:
            x, y = point[0]
            if y > height * 0.8:  # Consider points near the bottom of the frame
                if x < frame.shape[1] // 2:
                    leftX.append(x)
                else:
                    rightX.append(x)
                    
    if leftX and rightX:
        roadWidthPixels = np.mean(rightX) - np.mean(leftX)
        roadWidthMeters = roadWidthPixels * metersPerPixel
    else:
        roadWidthMeters = 0

    # Return the road width (in meters) and the greenMask to visualize the detected lines
    return roadWidthMeters, greenMask

# Video file path
videoPath = 'videos\\vid6.mp4'
cap = cv2.VideoCapture(videoPath)

# Get video dimensions and set meters per pixel
metersPerPixel = 0.021  # Your previously calculated value

# Define codec and create VideoWriter object to save the output video
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output_with_roadlines.avi', fourcc, 20.0, (640, 360))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Process each frame and get the road width in meters
    roadWidthMeters, greenMask = processFrame(frame, metersPerPixel)
    
    # Overlay the road width text on the frame
    cv2.putText(frame, f'Road Width: {roadWidthMeters:.2f} meters', (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
    
    # Combine the green mask with the original frame for better visualization
    combinedFrame = cv2.addWeighted(frame, 1, cv2.cvtColor(greenMask, cv2.COLOR_GRAY2BGR), 0.3, 0)

    # Show the frame with the road width
    cv2.imshow('Road Line Detection', combinedFrame)
    out.write(combinedFrame)  # Write the frame to the output video

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()